# Sorted Spikes Decoding

The mechanics of decoding with sorted spikes are largely similar to those of decoding with unsorted spikes. You should familiarize yourself with the [clusterless decoding tutorial](./42_Decoding_Clusterless.ipynb) before proceeding with this one.

The elements we will need to decode with sorted spikes are:
- `PositionGroup`
- `SortedSpikesGroup`
- `DecodingParameters`
- `encoding_interval`
- `decoding_interval`

This time, instead of extracting waveform features, we can proceed directly from the SpikeSortingOutput table to specify which units we want to decode. The rest of the decoding process is the same as before.


## SortedSpikesGroup

In [1]:
from pathlib import Path
import datajoint as dj

dj.config.load(
    Path("../dj_local_conf.json").absolute()
)  # load config for database connection info

In [2]:
from spyglass.spikesorting.spikesorting_merge import SpikeSortingOutput
import spyglass.spikesorting.v1 as sgs


nwb_copy_file_name = "mediumnwb20230802_.nwb"

sorter_keys = {
    "nwb_file_name": nwb_copy_file_name,
    "sorter": "clusterless_thresholder",
    "sorter_param_name": "default_clusterless",
}

(sgs.SpikeSortingSelection & sorter_keys) * SpikeSortingOutput.CurationV1

[2024-01-29 10:31:51,696][INFO]: Connecting root@localhost:3306
[2024-01-29 10:31:51,746][INFO]: Connected root@localhost:3306
[10:31:52][WARNING] Spyglass: Please update position_tools to >= 0.1.0


sorting_id,merge_id,recording_id,sorter,sorter_param_name,nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,curation_id
08a302b6-5505-40fa-b4d5-62162f8eef58,485a4ddf-332d-35b5-3ad4-0561736c1844,449b64e3-db0b-437e-a1b9-0d29928aa2dd,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,45f6b9a1-eef3-46eb-866d-d0999afebda6,0
0ca508ee-af4c-4a89-8181-d48bd209bfd4,6acb99b8-6a0c-eb83-1141-5f603c5895e0,328da21c-1d9c-41e2-9800-76b3484b707b,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,686d9951-1c0f-4d5e-9f5c-09e6fd8bdd4c,0
209dc048-6fae-4315-b293-c06fff29f947,f7237e18-4e73-4aee-805b-90735e9147de,aff78f2f-2ba0-412a-95cc-447c3a2f4683,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,719e8a86-fcf1-4ffc-8c1f-ea912f67ad5d,0
21a9a593-f6f3-4b82-99d7-8fc46556eff3,7e3fa66e-727e-1541-819a-b01309bb30ae,2402805a-04f9-4a88-9ccf-071376c8de19,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,d581b117-160e-4311-b096-7781a4de4394,0
406a20e3-5a9f-4fec-b046-a6561f72461e,6d039a63-17ad-0b78-4b1e-f02d5f3dbbc5,f1427e00-2974-4301-b2ac-b4dc29277c51,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,0e848c38-9105-4ea4-b6ba-dbdd5b46a088,0
4131c51b-c56d-41fa-b046-46635fc17fd9,e0e9133a-7a4e-1321-a43a-e8afcb2f25da,9e332d82-1daf-4e92-bb50-12e4f9430875,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,9ed11db5-c42e-491a-8caf-7d9a37a65f13,0
4c5a629a-71d9-481d-ab11-a4cb0fc16087,9959b614-2318-f597-6651-a3a82124d28a,3a2c3eed-413a-452a-83c8-0e4648141bde,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,2b9fbf14-74a0-4294-a805-26702340aac9,0
4d629c07-1931-4e1f-a3a8-cbf1b72161e3,c0eb6455-fc41-c200-b62e-e3ca81b9a3f7,f07bc0b0-de6b-4424-8ef9-766213aaca26,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,5c68f0f0-f577-4905-8a09-e4d171d0a22d,0
554a9a3c-0461-48be-8435-123eed59c228,912e250e-56d8-ee33-4525-c844d810971b,7f128981-6868-4976-ba20-248655dcac21,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,f4b9301f-bc91-455b-9474-c801093f3856,0
7bb007f2-26d3-463f-b7dc-7bd4d271725e,d7d2c97a-0e6e-d1b8-735c-d55dc66a30e1,a9b7cec0-1256-49cf-abf0-8c45fd155379,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,74270cba-36ee-4afb-ab50-2a6cc948e68c,0


In [3]:
spikesorting_merge_ids = (
    (sgs.SpikeSortingSelection & sorter_keys) * SpikeSortingOutput.CurationV1
).fetch("merge_id")

spikesorting_merge_ids

array([UUID('485a4ddf-332d-35b5-3ad4-0561736c1844'),
       UUID('6acb99b8-6a0c-eb83-1141-5f603c5895e0'),
       UUID('f7237e18-4e73-4aee-805b-90735e9147de'),
       UUID('7e3fa66e-727e-1541-819a-b01309bb30ae'),
       UUID('6d039a63-17ad-0b78-4b1e-f02d5f3dbbc5'),
       UUID('e0e9133a-7a4e-1321-a43a-e8afcb2f25da'),
       UUID('9959b614-2318-f597-6651-a3a82124d28a'),
       UUID('c0eb6455-fc41-c200-b62e-e3ca81b9a3f7'),
       UUID('912e250e-56d8-ee33-4525-c844d810971b'),
       UUID('d7d2c97a-0e6e-d1b8-735c-d55dc66a30e1'),
       UUID('abb92dce-4410-8f17-a501-a4104bda0dcf'),
       UUID('74e10781-1228-4075-0870-af224024ffdc'),
       UUID('8bbddc0f-d6ae-6260-9400-f884a6e25ae8'),
       UUID('614d796c-0b95-6364-aaa0-b6cb1e7bbb83'),
       UUID('b332482b-e430-169d-8ac0-0a73ce968ed7'),
       UUID('86897349-ff68-ac72-02eb-739dd88936e6'),
       UUID('4a712103-c223-864f-82e0-6c23de79cc14'),
       UUID('cf858380-e8a3-49de-c2a9-1a277e307a68'),
       UUID('cc4ee561-f974-f8e5-0ea4-83185263a

In [4]:
from spyglass.spikesorting.unit_inclusion_merge import (
    ImportedUnitInclusionV1,
    UnitInclusionOutput,
)

ImportedUnitInclusionV1().insert_all_units(spikesorting_merge_ids)

UnitInclusionOutput.ImportedUnitInclusionV1() & [
    {"spikesorting_merge_id": id} for id in spikesorting_merge_ids
]

merge_id,spikesorting_merge_id,unit_group_name Name of unit group
39b22078-84fa-d3aa-8ca9-2368603d6a47,0751a1e1-a406-7f87-ae6f-ce4ffc60621c,all units
7bc69ef8-ae89-f30f-c236-56a87a49ed63,485a4ddf-332d-35b5-3ad4-0561736c1844,all units
2f6295c6-ad14-dde0-6b14-ee29bc2ed849,4a712103-c223-864f-82e0-6c23de79cc14,all units
7fa3c0ad-36a8-c14c-d84a-890c6002a457,4a72c253-b3ca-8c13-e615-736a7ebff35c,all units
4b025478-ab06-8e65-ef09-7e32be5a624c,5c53bd33-d57c-fbba-e0fb-55e0bcb85d03,all units
9658d944-1f1f-d467-1f56-bcc32d63e6e8,614d796c-0b95-6364-aaa0-b6cb1e7bbb83,all units
76cb7574-a687-37c9-1045-8e4e5d13a0b0,6acb99b8-6a0c-eb83-1141-5f603c5895e0,all units
87af74fa-a77e-f29b-99f7-f5720aa4816f,6d039a63-17ad-0b78-4b1e-f02d5f3dbbc5,all units
3cccfce5-dad9-61c5-7eee-1c7081837100,74e10781-1228-4075-0870-af224024ffdc,all units
bbddf25f-d810-f586-e3b8-cb96350fbbde,7e3fa66e-727e-1541-819a-b01309bb30ae,all units


In [5]:
from spyglass.spikesorting.analysis.v1.group import SortedSpikesGroup

unit_inclusion_merge_ids = (
    UnitInclusionOutput.ImportedUnitInclusionV1
    & [{"spikesorting_merge_id": id} for id in spikesorting_merge_ids]
).fetch("merge_id")

SortedSpikesGroup().create_group(
    group_name="test_group",
    nwb_file_name=nwb_copy_file_name,
    unit_inclusion_merge_ids=unit_inclusion_merge_ids,
)

SortedSpikesGroup & {
    "nwb_file_name": nwb_copy_file_name,
    "sorted_spikes_group_name": "test_group",
}

nwb_file_name name of the NWB file,sorted_spikes_group_name
mediumnwb20230802_.nwb,test_group


In [6]:
SortedSpikesGroup.Units & {
    "nwb_file_name": nwb_copy_file_name,
    "sorted_spikes_group_name": "test_group",
}

nwb_file_name name of the NWB file,sorted_spikes_group_name,unit_inclusion_merge_id
mediumnwb20230802_.nwb,test_group,0363fb94-c5de-d653-ed33-1807fa523790
mediumnwb20230802_.nwb,test_group,12da7476-bb70-a4ee-0558-99ca5bcc0958
mediumnwb20230802_.nwb,test_group,151976b9-9dc6-74f6-65d3-23ce96d006d1
mediumnwb20230802_.nwb,test_group,1b7ba376-89f9-e0ac-9104-fbd1bd984e41
mediumnwb20230802_.nwb,test_group,2f6295c6-ad14-dde0-6b14-ee29bc2ed849
mediumnwb20230802_.nwb,test_group,357a0267-874e-04f0-410e-9711f29e1236
mediumnwb20230802_.nwb,test_group,39b22078-84fa-d3aa-8ca9-2368603d6a47
mediumnwb20230802_.nwb,test_group,3cccfce5-dad9-61c5-7eee-1c7081837100
mediumnwb20230802_.nwb,test_group,4b025478-ab06-8e65-ef09-7e32be5a624c
mediumnwb20230802_.nwb,test_group,7591b69d-a902-9058-7597-04632de9e08c


## Model parameters

As before we can specify the model parameters. The only difference is that we will use the `ContFragSortedSpikesClassifier` instead of the `ContFragClusterlessClassifier`.

In [7]:
from spyglass.decoding.v1.core import DecodingParameters
from non_local_detector.models import ContFragSortedSpikesClassifier


DecodingParameters.insert1(
    {
        "decoding_param_name": "contfrag_sorted",
        "decoding_params": ContFragSortedSpikesClassifier(),
        "decoding_kwargs": dict(),
    },
    skip_duplicates=True,
)

DecodingParameters()

decoding_param_name a name for this set of parameters,decoding_params initialization parameters for model,decoding_kwargs additional keyword arguments
contfrag_clusterless,=BLOB=,=BLOB=
contfrag_clusterless_0.5.13,=BLOB=,=BLOB=
contfrag_sorted,=BLOB=,=BLOB=
contfrag_sorted_0.5.13,=BLOB=,=BLOB=
nonlocal_clusterless_0.5.13,=BLOB=,=BLOB=
nonlocal_sorted_0.5.13,=BLOB=,=BLOB=


### 1D Decoding

As in the clusterless notebook, we can decode 1D position if we specify the `track_graph`, `edge_order`, and `edge_spacing` parameters in the `Environment` class constructor. See the [clusterless decoding tutorial](./42_Decoding_Clusterless.ipynb) for more details.

## Decoding

Now we can decode the position using the sorted spikes using the `SortedSpikesDecodingSelection` table. Here we assume that `PositionGroup` has been specified as in the clusterless decoding tutorial.

In [8]:
from spyglass.decoding.v1.sorted_spikes import SortedSpikesDecodingSelection

SortedSpikesDecodingSelection()

nwb_file_name name of the NWB file,sorted_spikes_group_name,position_group_name,decoding_param_name a name for this set of parameters,encoding_interval descriptive name of this interval list,decoding_interval descriptive name of this interval list,estimate_decoding_params whether to estimate the decoding parameters


In [9]:
selection_key = {
    "sorted_spikes_group_name": "test_group",
    "position_group_name": "test_group",
    "decoding_param_name": "contfrag_sorted",
    "nwb_file_name": "mediumnwb20230802_.nwb",
    "encoding_interval": "pos 0 valid times",
    "decoding_interval": "test decoding interval",
    "estimate_decoding_params": False,
}

SortedSpikesDecodingSelection.insert1(
    selection_key,
    skip_duplicates=True,
)

In [10]:
from spyglass.decoding.v1.sorted_spikes import SortedSpikesDecodingV1

SortedSpikesDecodingV1.populate(selection_key)

[10:31:56][WARNING] Spyglass: Upsampled position data, frame indices are invalid. Setting add_frame_ind=False
[2024-01-29 10:31:56,808][WARNING]: Skipped checksum for file with hash: 0cd40383-03e0-44ec-5dac-36c66063796a, and path: /Users/edeno/Documents/GitHub/spyglass/DATA/analysis/mediumnwb20230802/mediumnwb20230802_FUSH604NQA.nwb
[2024-01-29 10:31:57,259][WARNING]: Skipped checksum for file with hash: 148d9058-e6dc-e959-4c4d-75db9aa0b6e4, and path: /Users/edeno/Documents/GitHub/spyglass/DATA/analysis/mediumnwb20230802/mediumnwb20230802_EF6N6XI3AH.nwb
[2024-01-29 10:31:57,603][WARNING]: Skipped checksum for file with hash: b4b6404f-aaf8-c4cc-9abe-ceea56e103f3, and path: /Users/edeno/Documents/GitHub/spyglass/DATA/analysis/mediumnwb20230802/mediumnwb20230802_O7ZZ0F1XN7.nwb
[2024-01-29 10:31:57,956][WARNING]: Skipped checksum for file with hash: 4357905c-c6b9-3990-4d62-740a54cfc667, and path: /Users/edeno/Documents/GitHub/spyglass/DATA/analysis/mediumnwb20230802/mediumnwb20230802_X84BY

Encoding models:   0%|          | 0/23 [00:00<?, ?cell/s]

Non-Local Likelihood:   0%|          | 0/23 [00:00<?, ?cell/s]

/Users/edeno/miniconda3/envs/spyglass/lib/python3.9/site-packages/non_local_detector/models/base.py:780: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'state_bins' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates in the future (i.e., one coordinate for each multi-index level + one dimension coordinate). If you want to keep this behavior, you need to first wrap it explicitly using `mindex_coords = xarray.Coordinates.from_pandas_multiindex(mindex_obj, 'dim')` and pass it as coordinates, e.g., `xarray.Dataset(coords=mindex_coords)`, `dataset.assign_coords(mindex_coords)` or `dataarray.assign_coords(mindex_coords)`.
  results = xr.Dataset(
/Users/edeno/miniconda3/envs/spyglass/lib/python3.9/site-packages/xarray/namedarray/core.py:487: UserWarning: Duplicate dimension names present: dimensions {'states'} appear more than once in dims=('states', 'states'). We do not yet support duplicate dimension names, but we

We verify that the results have been inserted into the `DecodingOutput` merge table.

In [11]:
from spyglass.decoding.decoding_merge import DecodingOutput

DecodingOutput.SortedSpikesDecodingV1 & selection_key

merge_id,nwb_file_name name of the NWB file,sorted_spikes_group_name,position_group_name,decoding_param_name a name for this set of parameters,encoding_interval descriptive name of this interval list,decoding_interval descriptive name of this interval list,estimate_decoding_params whether to estimate the decoding parameters
650316f6-e19b-6195-5aea-0f46bc503fa3,mediumnwb20230802_.nwb,test_group,test_group,contfrag_sorted,pos 0 valid times,test decoding interval,0


We can load the results as before:

In [12]:
results = (SortedSpikesDecodingV1 & selection_key).fetch_results()
results

[2024-01-29 10:34:07,530][WARNING]: Skipped checksum for file with hash: 559cd050-31a1-1595-43a0-e86ff97adc73, and path: /Users/edeno/Documents/GitHub/spyglass/DATA/analysis/mediumnwb20230802/mediumnwb20230802_23fc2a6a-8c13-4256-bf78-8d2314a04709.nc
/Users/edeno/miniconda3/envs/spyglass/lib/python3.9/site-packages/xarray/namedarray/core.py:487: UserWarning: Duplicate dimension names present: dimensions {'states'} appear more than once in dims=('states', 'states'). We do not yet support duplicate dimension names, but we do allow initial construction of the object. We recommend you rename the dims immediately to become distinct, as most xarray functionality is likely to fail silently if you do not. To rename the dimensions you will need to set the ``.dims`` attribute of each variable, ``e.g. var.dims=('x0', 'x1')``.
  warnings.warn(
/Users/edeno/miniconda3/envs/spyglass/lib/python3.9/site-packages/xarray/namedarray/core.py:487: UserWarning: Duplicate dimension names present: dimensions {

<xarray.Dataset>
Dimensions:                      (state_ind: 26668, dim_0: 26668, time: 3750,
                                  states: 2, intervals: 1, state_bins: 26668)
Coordinates:
  * state_ind                    (state_ind) int32 0 0 0 0 0 0 0 ... 1 1 1 1 1 1
  * time                         (time) float64 1.626e+09 ... 1.626e+09
  * states                       (states) object 'Continuous' 'Fragmented'
    environments                 (states) object ...
    encoding_groups              (states) int32 ...
  * state_bins                   (state_bins) object MultiIndex
  * state                        (state_bins) object 'Continuous' ... 'Fragme...
  * x_position                   (state_bins) float64 29.02 29.02 ... 262.7
  * y_position                   (state_bins) float64 0.5211 2.516 ... 224.0
Dimensions without coordinates: dim_0, intervals
Data variables:
    initial_conditions           (dim_0) float64 ...
    discrete_state_transitions   (states, states) float64 ...
    acausal_posterior            (intervals, time, state_bins) float32 ...
    acausal_state_probabilities  (intervals, time, states) float64 ...
Attributes:
    marginal_log_likelihoods:  -22323.148